### s000 environment setup

In [0]:
# libraries
import os

import pyspark
from pyspark import sql
from pyspark.sql import functions as f
from pyspark.sql import Window

In [0]:
%run "./utils_spark_df"

In [0]:
%run "./utils_stratified_sampling"

In [0]:
dir_data_sfmc = "/mnt/prod_sfmc/imports/DataAnalytics/"
#df_check_1 = spark.read.options(header=True).csv(f"{dir_data_sfmc}/250305-RM-MOBPM-Wallet-Samsung-Galaxy-SEND-S24-WASH.csv")
df_check_s24 = spark.read.options(header = True).csv(f"{dir_data_sfmc}/250305-RM-MOBPM-Wallet-Samsung-Galaxy-SEND-S24-WASHV2.csv")
df_check_s24fe = spark.read.options(header = True).csv(f"{dir_data_sfmc}/250305-RM-MOBPM-Wallet-Samsung-Galaxy-SEND-S24FE-WASH.csv")
#250305-RM-MOBPM-Wallet-Samsung-Galaxy-SEND-S24FE-WASH


df_s24_campaign = spark.read.format('delta').load('/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/wallet/20250305_samsungs24')
df_s24fe_campaign_control = spark.read.format('delta').load('/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/wallet/20250305_samsungs24fe_control')
df_s24fe_campaign_target = spark.read.format('delta').load('/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/wallet/20250305_samsungs24fe_target')

# ifp 
df_prm_ifp_bill = spark.read.format('delta').load('/mnt/feature-store-prod-lab/d300_primary/d301_mobile_oa_consumer/prm_ifp_main_on_bill')
df_prm_ifp_srvc = spark.read.format('delta').load('dbfs:/mnt/feature-store-prod-lab/d300_primary/d301_mobile_oa_consumer/prm_ifp_main_on_service')

In [0]:
df_ifp_bill = (
    df_prm_ifp_bill
    .filter(f.col('ifp_type') == 'device')
    .filter(f.col('ifp_term_start_date').isNotNull())
    .select('fs_cust_id', 'fs_acct_id', 'fs_ifp_id', 'ifp_model', 'ifp_term_start_date', 'ifp_term_end_date', 'ifp_sales_channel', 'ifp_sales_channel_branch', 'ifp_sales_channel_group')
    .distinct()
)

df_ifp_srvc=(
    df_prm_ifp_srvc
    .filter(f.col('ifp_type') == 'device')
    .filter(f.col('ifp_term_start_date').isNotNull())
    .select('fs_cust_id', 'fs_acct_id', 'fs_ifp_id', 'ifp_model','ifp_term_start_date', 'ifp_term_end_date', 'ifp_sales_channel', 'ifp_sales_channel_branch', 'ifp_sales_channel_group')
    .distinct()
)

df_ifp_all=(
    df_ifp_bill
    .union(df_ifp_srvc)
)

In [0]:
vt_send_date = '2025-03-05' # send date = ''

In [0]:
display(
    df_s24_campaign
    .filter(f.col('campaign_cohort') == 'Local Control')
)


display(
    df_check_s24
    .limit(10)
        
)

display(
    df_check_s24
    .count()
)

In [0]:
display(
    df_check_s24
    .join(
        df_s24_campaign
        , (f.col('fs_cust_id') == f.col('Contact_Key'))
            & (f.col('fs_acct_id') == f.col('Billing_Account_Number'))
            & (f.col('fs_srvc_id') == f.col('Service_ID'))
        , 'inner'
    )
    .agg(
       f.count("*"),
        f.countDistinct("fs_srvc_id"),
        f.median("churn_score"),
        f.avg("churn_score"),
        f.median("dr_score"),
        f.avg("dr_score"),
        f.median("ifp_score"),
        f.avg("ifp_score")
    )
)


display(
    df_s24_campaign
    .filter(f.col('campaign_cohort') == 'Local Control')
    .agg(
       f.count("*"),
        f.countDistinct("fs_srvc_id"),
        f.median("churn_score"),
        f.avg("churn_score"),
        f.median("dr_score"),
        f.avg("dr_score"),
        f.median("ifp_score"),
        f.avg("ifp_score")
    )
)


In [0]:
df_s24_sent =(
    df_check_s24
    .join(
        df_s24_campaign
        .drop('offer')
        , (f.col('fs_cust_id') == f.col('Contact_Key'))
            & (f.col('fs_acct_id') == f.col('Billing_Account_Number'))
            & (f.col('fs_srvc_id') == f.col('Service_ID'))
        , 'inner'
    )
)

display(df_check_s24.count())
display(df_s24_sent.count())

In [0]:
display(
    df_s24_sent
    .join(df_ifp_all, ['fs_cust_id', 'fs_acct_id'], 'inner')
    .filter(f.col('ifp_term_start_date') >= '2025-03-05')
    .filter(f.col('ifp_term_start_date') <= '2025-03-31')
    .groupBy('ifp_model')
    .agg(f.countDistinct('fs_acct_id')
         , f.countDistinct('fs_ifp_id')
    )
)


In [0]:
display(
    df_s24_sent
    .join(df_ifp_all, ['fs_cust_id', 'fs_acct_id'], 'inner')
    .filter(f.col('ifp_term_start_date') >= '2025-03-05')
    .filter(f.col('ifp_term_start_date') <= '2025-03-31')
    .filter(f.col('ifp_model') == 'Samsung Galaxy S24')
    .groupBy('offer')
    .agg(
        f.countDistinct('fs_acct_id')
        , f.countDistinct('fs_ifp_id')
    )
)

display(
    df_s24_sent
    .select('fs_cust_id', 'fs_acct_id', 'fs_srvc_id',  'ifp_segment', 'dr_segment', 'offer')
    .groupBy('offer')
    .agg(f.countDistinct('fs_srvc_id'))
    
)



### S24FE campaign

In [0]:
df_s24fe_sent =(
    df_check_s24fe
    .join(
        df_s24fe_campaign_target
        , (f.col('fs_cust_id') == f.col('Contact_Key'))
            & (f.col('fs_acct_id') == f.col('Billing_Account_Number'))
            & (f.col('fs_srvc_id') == f.col('Service_ID'))
        , 'inner'
    )
)

display(df_check_s24fe.count())
display(df_s24fe_sent.count())

In [0]:
display(
    df_s24fe_sent
    .join(df_ifp_all, ['fs_cust_id', 'fs_acct_id'], 'inner')
    .filter(f.col('ifp_term_start_date') >= '2025-03-10')
    .filter(f.col('ifp_term_start_date') <= '2025-03-31')
    .groupBy('ifp_model')
    .agg(
        f.countDistinct('fs_acct_id')
        , f.countDistinct('fs_ifp_id')
    )
)


In [0]:
display(
    df_s24fe_sent
    .join(df_ifp_all, ['fs_cust_id', 'fs_acct_id'], 'inner')
    .filter(f.col('ifp_term_start_date') >= '2025-03-10')
    .filter(f.col('ifp_term_start_date') <= '2025-03-31')
    .filter(f.col('ifp_model') == 'Samsung Galaxy S24 FE')
    .groupBy('offer')
    .agg(
        f.countDistinct('fs_acct_id')
        , f.countDistinct('fs_ifp_id')
    )
)

display(
    df_s24fe_sent
    .select('fs_cust_id', 'fs_acct_id', 'fs_srvc_id',  'ifp_segment', 'dr_segment', 'offer')
    .groupBy('offer')
    .agg(f.countDistinct('fs_srvc_id'))
    
)



### regenerate control S24

In [0]:
df_campaign_full = spark.read.format("delta").load("/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/wallet/20250305_samsung_cofound")

In [0]:
vt_param_control_size = 35000

ls_param_strata = [ "churn_top_ntile","ifp_top_ntile", "dr_top_ntile"]

df_sample_target = create_sample_target(
    df_s24_sent
    , ls_param_strata
    )

df_campaign_cand_control = find_similar_sample(
    df_campaign_full
    .join(df_s24_sent, ['fs_acct_id', 'fs_cust_id', 'fs_srvc_id'], 'anti')
    .filter(f.col("network_dvc_brand_std") != 'apple')
    #.filter(f.col("ch01_flag") != 'Y')
    #.filter(f.col('wallence_bal_category')<=500)
    .filter(
        ~((f.col('ifp_segment') == 'L')
          & (f.col('dr_segment') == 'L')
        ) 
    )
    .join(df_s24fe_sent, ['fs_acct_id', 'fs_cust_id', 'fs_srvc_id'], 'anti')
    , size = vt_param_control_size
    , strata = ls_param_strata
    , df_target = df_sample_target
)

print("control")
display(
    df_campaign_cand_control
    .agg(
        f.countDistinct("fs_srvc_id")
        , f.median("churn_score")
        , f.avg("churn_score")
        , f.median("dr_score")
        , f.avg("dr_score")
        , f.median("ifp_score")
        , f.avg("ifp_score")
    )
)


print("target")
display(
    df_s24_sent
    .agg(
        f.countDistinct("fs_srvc_id")
        , f.median("churn_score")
        , f.avg("churn_score")
        , f.median("dr_score")
        , f.avg("dr_score")
        , f.median("ifp_score")
        , f.avg("ifp_score")
    )
)

evaluate_sample(df_campaign_cand_control, df_s24_sent,['dr_top_ntile', 'ifp_top_ntile', 'churn_top_ntile'] )

(
    df_campaign_cand_control
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    #.option("partitionOverwriteMode", "dynamic")
    .save('/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/regen_control/20250408_SamsungS24_regen_control')  
)

df_campaign_cand_control = spark.read.format('delta').load('/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/regen_control/20250408_SamsungS24_regen_control')

In [0]:
display(df_s24_sent
        .join(df_campaign_cand_control, ['fs_acct_id', 'fs_cust_id', 'fs_srvc_id'], 'inner')      
        )



### calculate conversion on control - S24

In [0]:
display(
    df_campaign_cand_control
    .join(df_ifp_all, ['fs_cust_id', 'fs_acct_id'], 'inner')
    .filter(f.col('ifp_term_start_date') >= f.lit(vt_send_date))
    .filter(f.col('ifp_term_start_date') <= '2025-03-31')
    .groupBy('ifp_model')
    .agg(
        f.countDistinct('fs_acct_id')
        , f.countDistinct('fs_ifp_id') 
    )
)


### regenerate control S24FE

In [0]:
vt_param_control_size = 27000

ls_param_strata = [ "churn_top_ntile","ifp_top_ntile", "dr_top_ntile"]

df_sample_target = create_sample_target(
    df_s24fe_sent
    , ls_param_strata
    )

df_campaign_cand_control = find_similar_sample(
    df_campaign_full
    .join(df_s24fe_sent, ['fs_acct_id', 'fs_cust_id', 'fs_srvc_id'], 'anti')
    .filter(f.col("network_dvc_brand_std") != 'apple')
    #.filter(f.col("ch01_flag") != 'Y')
    .filter(f.col('wallence_bal_category')<=200)
    .filter(f.col('propensity_combine') == 'H/M')
    .join(df_s24_sent, ['fs_acct_id', 'fs_cust_id', 'fs_srvc_id'], 'anti')
    , size = vt_param_control_size
    , strata = ls_param_strata
    , df_target = df_sample_target
)

print("control")
display(
    df_campaign_cand_control
    .agg(
        f.countDistinct("fs_srvc_id")
        , f.median("churn_score")
        , f.avg("churn_score")
        , f.median("dr_score")
        , f.avg("dr_score")
        , f.median("ifp_score")
        , f.avg("ifp_score")
    )
)


print("target")
display(
    df_s24fe_sent
    .agg(
        f.countDistinct("fs_srvc_id")
        , f.median("churn_score")
        , f.avg("churn_score")
        , f.median("dr_score")
        , f.avg("dr_score")
        , f.median("ifp_score")
        , f.avg("ifp_score")
    )
)

evaluate_sample(df_campaign_cand_control, df_s24fe_sent,['dr_top_ntile', 'ifp_top_ntile', 'churn_top_ntile'] )

(
    df_campaign_cand_control
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    #.option("partitionOverwriteMode", "dynamic")
    .save('/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/regen_control/20250408_SamsungS24FE_regen_control')  
)

df_campaign_cand_control = spark.read.format('delta').load('/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/regen_control/20250408_SamsungS24FE_regen_control')

In [0]:
display(
    df_campaign_cand_control
    .join(df_ifp_all, ['fs_cust_id', 'fs_acct_id'], 'inner')
    .filter(f.col('ifp_term_start_date') >= '2025-03-14')
    .filter(f.col('ifp_term_start_date') <= '2025-03-31')
    .groupBy('ifp_model')
    .agg(
        f.countDistinct('fs_acct_id')
        , f.countDistinct('fs_ifp_id') 
    )
)

display(
    df_s24fe_sent
    .join(df_ifp_all, ['fs_cust_id', 'fs_acct_id'], 'inner')
    .filter(f.col('ifp_term_start_date') >= '2025-03-14')
    .filter(f.col('ifp_term_start_date') <= '2025-03-31')
    .groupBy('ifp_model')
    .agg(
        f.countDistinct('fs_acct_id')
        , f.countDistinct('fs_ifp_id')
    )
)

